# Calculate monthly mean data from daily inputs

## notes

## imports

In [14]:
%%time
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import os
import xarray as xr
import xcdat as xc

CPU times: user 30 µs, sys: 0 ns, total: 30 µs
Wall time: 33.1 µs


## set data paths

In [2]:
obsPath = "/p/user_pub/PCMDIobs/obs4MIPs_input/RSS/RSS-MW5-1/v20230605/"

## lazy load data

In [3]:
def setCalendar(ds):
    # https://github.com/pydata/xarray/issues/6259
    #ds.time.calendar = "standard"
    #ds["time"].calendar
    ds.time.attrs['calendar'] = 'standard'
    return xr.decode_cf(ds)

dataPath = os.path.join(obsPath, "199801*.nc")
print("dataPath:", dataPath)
ds = xc.open_mfdataset(dataPath) #, preprocess=setCalendar)

dataPath: /p/user_pub/PCMDIobs/obs4MIPs_input/RSS/RSS-MW5-1/v20230605/199801*.nc


2023-11-22 15:34:25,562 [WARNING]: dataset.py(decode_time:402) >> 'time' does not have a calendar attribute set. Defaulting to CF 'standard' calendar.
2023-11-22 15:34:25,562 [WARNING]: dataset.py(decode_time:402) >> 'time' does not have a calendar attribute set. Defaulting to CF 'standard' calendar.
2023-11-22 15:34:25,572 [WARNING]: dataset.py(decode_time:402) >> 'time' does not have a calendar attribute set. Defaulting to CF 'standard' calendar.
2023-11-22 15:34:25,572 [WARNING]: dataset.py(decode_time:402) >> 'time' does not have a calendar attribute set. Defaulting to CF 'standard' calendar.
2023-11-22 15:34:25,580 [WARNING]: dataset.py(decode_time:402) >> 'time' does not have a calendar attribute set. Defaulting to CF 'standard' calendar.
2023-11-22 15:34:25,580 [WARNING]: dataset.py(decode_time:402) >> 'time' does not have a calendar attribute set. Defaulting to CF 'standard' calendar.
2023-11-22 15:34:25,589 [WARNING]: dataset.py(decode_time:402) >> 'time' does not have a calen

## view dataset

In [4]:
ds

<xarray.Dataset>
Dimensions:           (lat: 720, lon: 1440, time: 31, bnds: 2)
Coordinates:
  * lat               (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon               (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time              (time) object 1998-01-01 12:00:00 ... 1998-01-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    lon_bnds          (lon, bnds) float32 -180.0 -179.8 -179.8 ... 179.8 180.0
    lat_bnds          (lat, bnds) float32 -90.0 -89.75 -89.75 ... 89.75 90.0
Attributes: (12/46)
    Conventions:                CF-1.8,ACDD-1.3
    title:                      Analysed foundation sea surface temperature o...
    summary:                    A merged, multi-sensor L4 foundation SST prod...
    references:                 http://www.remss.com/measurements/sea-surface...
    institution:                REMSS
    history:                    2021-11-11 18:18:15+0000 created by sst_fusio...
    ...                         ...
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_url:              http://www.ghrsst.org
    processing_level:           L4
    cdm_data_type:              grid

## add calendar attribute to time axis

In [5]:
ds.time.attrs['calendar'] = 'standard'

In [6]:
ds

<xarray.Dataset>
Dimensions:           (lat: 720, lon: 1440, time: 31, bnds: 2)
Coordinates:
  * lat               (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon               (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time              (time) object 1998-01-01 12:00:00 ... 1998-01-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    lon_bnds          (lon, bnds) float32 -180.0 -179.8 -179.8 ... 179.8 180.0
    lat_bnds          (lat, bnds) float32 -90.0 -89.75 -89.75 ... 89.75 90.0
Attributes: (12/46)
    Conventions:                CF-1.8,ACDD-1.3
    title:                      Analysed foundation sea surface temperature o...
    summary:                    A merged, multi-sensor L4 foundation SST prod...
    references:                 http://www.remss.com/measurements/sea-surface...
    institution:                REMSS
    history:                    2021-11-11 18:18:15+0000 created by sst_fusio...
    ...                         ...
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_url:              http://www.ghrsst.org
    processing_level:           L4
    cdm_data_type:              grid

## add missing time_bnds

In [7]:
ds = ds.bounds.add_missing_bounds(axes="T")
#ds.time.attrs["bounds"] = "time_bnds"

In [8]:
ds

<xarray.Dataset>
Dimensions:           (lat: 720, lon: 1440, time: 31, bnds: 2)
Coordinates:
  * lat               (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon               (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time              (time) object 1998-01-01 12:00:00 ... 1998-01-31 12:00:00
Dimensions without coordinates: bnds
Data variables:
    analysed_sst      (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    analysis_error    (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    sea_ice_fraction  (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    mask              (time, lat, lon) float32 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
    lon_bnds          (lon, bnds) float32 -180.0 -179.8 -179.8 ... 179.8 180.0
    lat_bnds          (lat, bnds) float32 -90.0 -89.75 -89.75 ... 89.75 90.0
    time_bnds         (time, bnds) object 1998-01-01 00:00:00 ... 1998-02-01 ...
Attributes: (12/46)
    Conventions:                CF-1.8,ACDD-1.3
    title:                      Analysed foundation sea surface temperature o...
    summary:                    A merged, multi-sensor L4 foundation SST prod...
    references:                 http://www.remss.com/measurements/sea-surface...
    institution:                REMSS
    history:                    2021-11-11 18:18:15+0000 created by sst_fusio...
    ...                         ...
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_url:              http://www.ghrsst.org
    processing_level:           L4
    cdm_data_type:              grid

In [9]:
mean_monthXr = ds.analysed_sst.resample(time='M').mean()
# https://stackoverflow.com/questions/50564459/using-xarray-to-make-monthly-average

In [25]:
mean_monthXr

<xarray.DataArray 'analysed_sst' (time: 1, lat: 720, lon: 1440)>
dask.array<stack, shape=(1, 720, 1440), dtype=float32, chunksize=(1, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time     (time) object 1998-01-31 00:00:00
Attributes:
    units:                  K
    long_name:              analysed sea surface temperature
    standard_name:          sea_surface_foundation_temperature
    valid_min:              -32767
    valid_max:              32767
    source:                 REMSS-L3C-TMI
    coverage_content_type:  physicalMeasurement

In [10]:
mean_monthXc = ds.temporal.group_average("analysed_sst", freq="month", weighted=True)

In [11]:
mean_monthXc

<xarray.Dataset>
Dimensions:       (lat: 720, lon: 1440, bnds: 2, time: 1)
Coordinates:
  * lat           (lat) float32 -89.88 -89.62 -89.38 ... 89.38 89.62 89.88
  * lon           (lon) float32 -179.9 -179.6 -179.4 ... 179.4 179.6 179.9
  * time          (time) object 1998-01-01 00:00:00
Dimensions without coordinates: bnds
Data variables:
    lon_bnds      (lon, bnds) float32 -180.0 -179.8 -179.8 ... 179.8 179.8 180.0
    lat_bnds      (lat, bnds) float32 -90.0 -89.75 -89.75 ... 89.75 89.75 90.0
    analysed_sst  (time, lat, lon) float64 dask.array<chunksize=(1, 720, 1440), meta=np.ndarray>
Attributes: (12/46)
    Conventions:                CF-1.8,ACDD-1.3
    title:                      Analysed foundation sea surface temperature o...
    summary:                    A merged, multi-sensor L4 foundation SST prod...
    references:                 http://www.remss.com/measurements/sea-surface...
    institution:                REMSS
    history:                    2021-11-11 18:18:15+0000 created by sst_fusio...
    ...                         ...
    project:                    Group for High Resolution Sea Surface Tempera...
    publisher_name:             The GHRSST Project Office
    publisher_email:            ghrsst-po@nceo.ac.uk
    publisher_url:              http://www.ghrsst.org
    processing_level:           L4
    cdm_data_type:              grid

In [12]:
ds.analysed_sst

<xarray.DataArray 'analysed_sst' (time: 31, lat: 720, lon: 1440)>
dask.array<concatenate, shape=(31, 720, 1440), dtype=float32, chunksize=(1, 720, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float32 -89.88 -89.62 -89.38 -89.12 ... 89.38 89.62 89.88
  * lon      (lon) float32 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time     (time) object 1998-01-01 12:00:00 ... 1998-01-31 12:00:00
Attributes:
    units:                  K
    long_name:              analysed sea surface temperature
    standard_name:          sea_surface_foundation_temperature
    valid_min:              -32767
    valid_max:              32767
    source:                 REMSS-L3C-TMI
    coverage_content_type:  physicalMeasurement